# 카카오 API를 활용한 지오코딩: 주소에서 GeoPackage로 변환하기

In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [2]:
# 카카오 API KEY
API_KEY = "2b6a0d9b90f56fb12baf4d2a23c937bb"

In [3]:
# 주소를 좌표로 변환
def convert_address_to_coordinates(address):
    api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}

    params = {"query": address}
    
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            coordinates = result["documents"][0]["y"], result["documents"][0]["x"]
            print(result)
            return coordinates
        else:
            print(result)
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [13]:
addr = '서울특별시 종로구 옥인동'
addr2 = '경기도 광주시 오포읍 능평리'
addr3 = '경기도 고양시 일산동구 식사동'

In [14]:
addr_return = convert_address_to_coordinates(addr)
print(addr_return)

{'documents': [{'address': {'address_name': '서울 종로구 옥인동', 'b_code': '1111011100', 'h_code': '', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '종로구', 'region_3depth_h_name': '', 'region_3depth_name': '옥인동', 'sub_address_no': '', 'x': '126.963385995622', 'y': '37.5835230926996'}, 'address_name': '서울 종로구 옥인동', 'address_type': 'REGION', 'road_address': None, 'x': '126.963385995622', 'y': '37.5835230926996'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
('37.5835230926996', '126.963385995622')


In [6]:
addr_return = convert_address_to_coordinates(addr2)
print(addr_return)

{'documents': [{'address': {'address_name': '경기 광주시 능평동', 'b_code': '4161011600', 'h_code': '4161061000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '광주시', 'region_3depth_h_name': '능평동', 'region_3depth_name': '능평동', 'sub_address_no': '', 'x': '127.167264087448', 'y': '37.3497062329604'}, 'address_name': '경기 광주시 능평동', 'address_type': 'REGION', 'road_address': None, 'x': '127.167264087448', 'y': '37.3497062329604'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
('37.3497062329604', '127.167264087448')


In [7]:
addr_return = convert_address_to_coordinates(addr3)
print(addr_return)

{'documents': [{'address': {'address_name': '경기 고양시 일산동구 식사동', 'b_code': '4128510100', 'h_code': '4128551000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '고양시 일산동구', 'region_3depth_h_name': '식사동', 'region_3depth_name': '식사동', 'sub_address_no': '', 'x': '126.813707127491', 'y': '37.6773400834299'}, 'address_name': '경기 고양시 일산동구 식사동', 'address_type': 'REGION', 'road_address': None, 'x': '126.813707127491', 'y': '37.6773400834299'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
('37.6773400834299', '126.813707127491')


In [8]:
# 데이터프레임에 좌표 추가
def add_coordinates_to_dataframe(df, address_column):
    latitudes = []
    longitudes = []

    for address in df[address_column]:
        coordinates = convert_address_to_coordinates(address)
        if coordinates:
            latitudes.append(coordinates[0])
            longitudes.append(coordinates[1])
        else:
            latitudes.append(None)
            longitudes.append(None)

    df["decimalLatitude"] = latitudes
    df["decimalLongitude"] = longitudes

In [9]:
# 데이터프레임
data = {'Name': ['국립공원공단 본사', '국립공원연구원 본원', '치악산국립공원사무소'],
        'Address': ['강원특별자치도 원주시 혁신로 22', '강원특별자치도 원주시 단구로 171', '강원특별자치도 원주시 소초면 무쇠점2길 26']}
df = pd.DataFrame(data)
df.head()

,Name,Address
0,국립공원공단 본사,강원특별자치도 원주시 혁신로 22
1,국립공원연구원 본원,강원특별자치도 원주시 단구로 171
2,치악산국립공원사무소,강원특별자치도 원주시 소초면 무쇠점2길 26


In [10]:
# 주소를 좌표로 변환하여 데이터프레임에 추가
add_coordinates_to_dataframe(df, 'Address')
df.to_csv('korea_np.csv', index=False, encoding='utf-8')
df.head()

{'documents': [{'address': {'address_name': '강원특별자치도 원주시 반곡동 2049-4', 'b_code': '5113011700', 'h_code': '5113067500', 'main_address_no': '2049', 'mountain_yn': 'N', 'region_1depth_name': '강원특별자치도', 'region_2depth_name': '원주시', 'region_3depth_h_name': '반곡관설동', 'region_3depth_name': '반곡동', 'sub_address_no': '4', 'x': '127.976704931245', 'y': '37.3238242077259'}, 'address_name': '강원특별자치도 원주시 혁신로 22', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '강원특별자치도 원주시 혁신로 22', 'building_name': '국립공원관리공단', 'main_building_no': '22', 'region_1depth_name': '강원특별자치도', 'region_2depth_name': '원주시', 'region_3depth_name': '반곡동', 'road_name': '혁신로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.976704931245', 'y': '37.3238242077259', 'zone_no': '26466'}, 'x': '127.976704931245', 'y': '37.3238242077259'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
{'documents': [{'address': {'address_name': '강원특별자치도 원주시 명륜동 242-2', 'b_code': '5113010600', 'h_code': '5113054100'

,Name,Address,decimalLatitude,decimalLongitude
0,국립공원공단 본사,강원특별자치도 원주시 혁신로 22,37.3238242077259,127.976704931245
1,국립공원연구원 본원,강원특별자치도 원주시 단구로 171,37.3369694079522,127.947724482633
2,치악산국립공원사무소,강원특별자치도 원주시 소초면 무쇠점2길 26,37.4140421500777,128.050384626252


In [11]:
# CSV를 GeoPackage로 변환
def csv_to_gpkg(input_csv, output_gpkg):
    df = pd.read_csv(input_csv)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.decimalLongitude, df.decimalLatitude),
        crs="EPSG:4326",
    )
    gdf.to_file(output_gpkg, driver="GPKG")
    print("데이터 변환 및 저장 완료:", output_gpkg)

In [12]:
input_csv = "korea_np.csv"
output_gpkg = "korea_np.gpkg"

csv_to_gpkg(input_csv, output_gpkg)

데이터 변환 및 저장 완료: korea_np.gpkg
